### 1. Import the physics, base model and simulation model into workspace

In [ ]:
from darts.models.darts_model import DartsModel
from darts.models.reservoirs.struct_reservoir import StructReservoir
from darts.models.physics.geothermal_g import Geothermal
from darts.engines import value_vector
import numpy as np

### 2. Build Model class
#### Brief Introduction of model inheritance
* Here create the <font color='red'>'Model' </font>  class, which inherits from <font color='red'>DartsModel</font> (the base class).
* It keeps all the functionalities of <font color='red'>DartsModel</font> and can also be extended to add more functionalities.
* If a function is redefined in subclass, the function in base class with identical name will be overridden (just like C++).

In [ ]:
class Model(DartsModel):
    def __init__(self):
        # call base class constructor
        super().__init__()

        self.timer.node["initialization"].start()

        # predefined reservoir size: 60*40*1
        self.nx = 500
        self.ny = 1
        self.n_up = 1
        self.n_bot = 1
        self.nz = 10 + self.n_up + self.n_bot
        self.up_bs = self.n_up * self.nx * self.ny
        self.bot_bs= self.n_bot* self.nx * self.ny
        self.nb = self.nx * self.ny * self.nz
        self.kx = np.ones(self.nb)
        self.kz = np.ones(self.nb)
        self.ky = 1
        self.dz = np.ones(self.nb)
        self.depth = np.ones(self.nb)
        dz = np.ones(self.nz) * 10
        dz[0] = 800
        dz[-1]= 800
        permx = [0.01, 208, 36.9, 23.3, 162, 82, 198, 12.7, 187, 22.2, 9.2, 0.01]
        permz = [0.01, 208, 36.9, 23.3, 162, 82, 198, 12.7, 187, 22.2, 9.2, 0.01]
        

        for z in range(self.nz):
            self.kx[self.nx*z:self.nx*(z+1)] = permx[z]
            self.kz[self.nx*z:self.nx*(z+1)] = permx[z] / 10
            self.dz[self.nx*z:self.nx*(z+1)] = dz[z]
        
        # top of the payzone is 2500m based on the thesis
        self.depth[:self.up_bs] = 2500 - dz[0] / 2  # depth of the overburden
        self.depth[self.up_bs:self.up_bs+self.nx] = 2500 + dz[1] / 2 # depth of the 1st layer of the payzone
        for z in range(2, self.nz):
            self.depth[self.up_bs+self.nx*(z-1):self.up_bs+self.nx*(z)] = \
                                                self.depth[self.up_bs+self.nx*(z-2):self.up_bs+self.nx*(z-1)] + dz[z] 
            
        self.poro = 0.2
        
        # Create reservoir using StructReservoir.
        # Just pass-in the necessary parameters and a box reservoir is generated.
        self.reservoir = StructReservoir(self.timer, nx=self.nx, ny=self.ny, nz=self.nz, dx=1, dy=100,
                                         dz=self.dz, permx=self.kx, permy=self.ky, permz=self.kz,
                                         poro=self.poro, depth=self.depth)
        # Get the number of reservoir grids
        self.nb = self.reservoir.mesh.n_res_blocks

        # Create numpy arrays wrapped around mesh data (no copying)
        self.volume = np.array(self.reservoir.mesh.volume, copy=False)
        self.hcap = np.array(self.reservoir.mesh.heat_capacity, copy=False)
        self.cond = np.array(self.reservoir.mesh.rock_cond, copy=False)

        self.volume[0] = 1e9

        # Constant definitions
        self.cond.fill(2*86.4)
        self.hcap.fill(4814)

        # add wells
        n_perf = self.nz
        self.reservoir.add_well("I1")
        for n in range(n_perf):
            self.reservoir.add_perforation(well=self.reservoir.wells[-1], i=1, j=1, k=n+1, well_index=100,
                                           multi_segment=False, verbose=True)

        self.reservoir.add_well("P1")
        for n in range(n_perf):
            self.reservoir.add_perforation(well=self.reservoir.wells[-1], i=self.nx, j=1, k=n+1, well_index=100,
                                           multi_segment=False, verbose=True)

        # add predefined physics
        self.physics = Geothermal(self.timer, 128, 0, 500, 100, 20000)

        # time step setting
        self.params.first_ts = 1e-4
        self.params.mult_ts = 2
        self.params.max_ts = 100

        # Newton tolerance is relatively high because of L2-norm for residual and well segments
        self.params.tolerance_newton = 1e-3
        self.params.tolerance_linear = 1e-5
        self.params.max_i_newton = 20
        self.params.max_i_linear = 50

        # default runtime
        self.runtime = 1000
        self.timer.node["initialization"].stop()

    '''Give initial pressure and temperature conditions to reservoir'''

    def set_initial_conditions(self):
        # set uniform initial condition
        #self.physics.set_uniform_initial_conditions(self.reservoir.mesh, uniform_pressure=250, \
        #                                            uniform_temperature=273.15 + 90)
        # set non-uniform initial condition
        self.physics.set_nonuniform_initial_conditions(self.reservoir.mesh, pressure_grad=100, \
                                                    temperature_grad=30)

    '''Give the well controls'''

    def set_boundary_conditions(self):
        for i, w in enumerate(self.reservoir.wells):
            if i == 0:
                # w.control = self.physics.new_bhp_water_inj(300, 308.15)
                w.control = self.physics.new_rate_water_inj(150*24, 308.15)
                w.constraint = self.physics.new_bhp_water_inj(450, 308.15)
            else:
                # w.control = self.physics.new_bhp_prod(200)
                w.control = self.physics.new_rate_water_prod(150*24)
                w.constraint = self.physics.new_bhp_prod(100)

    def set_rate(self, rate, welln, temp=308):
        w = self.reservoir.wells[welln]
        if welln == 0:
            w.control = self.physics.new_rate_water_inj(rate, temp)
        else:
            w.control = self.physics.new_rate_water_prod(rate)

    def set_bhp(self, bhp, welln, temp=308):
        w = self.reservoir.wells[welln]
        if welln == 0:
            w.control = self.physics.new_bhp_inj(bhp, temp)
        else:
            w.control = self.physics.new_bhp_prod(bhp)

### 3. Create and initialize simulation model

In [ ]:
# build a Model object
m = Model()
m.init()

### 4. Run simulation and print statistics

In [ ]:
m.run_python()
m.print_timers()

### 5. Data process and plot

In [ ]:
%matplotlib inline

import pandas as pd
# access to engine time-dependent data
time_data = pd.DataFrame.from_dict(m.physics.engine.time_data)
# wirte timedata to output file
time_data.to_pickle("darts_time_data.pkl")
# write timedata to excel file
writer = pd.ExcelWriter('time_data.xlsx')
time_data.to_excel(writer, 'Sheet1')
writer.save()

# read data from output file
time_data = pd.read_pickle("darts_time_data.pkl")

from darts.tools.plot_darts import *
# plot production temperature
p_w = 'P1'
ax = plot_water_rate_darts(p_w, time_data)
ax = plot_temp_darts(p_w, time_data)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from darts.models.physics.iapws.iapws_property_vec import _Backward1_T_Ph_vec

# access to engine solution
X = np.array(m.physics.engine.X, copy=False)
pressure  = X[0::2][:m.reservoir.nb]
enthalpy  = X[1::2][:m.reservoir.nb]
temp = _Backward1_T_Ph_vec(pressure/10, enthalpy/18.015)  # in Kelvin

In [ ]:
nx = m.reservoir.nx
nz = m.reservoir.nz - m.n_up - m.n_bot
nb = m.reservoir.nb

plt.figure(figsize=(15,3))
X, Y  = np.meshgrid(range(0, nx), range(0, nz+1))
X = X
Y = Y * 10
# plot the results of payzone
t_payzone = temp[m.up_bs : nb-m.bot_bs]
surf = plt.pcolormesh(X, Y, t_payzone.reshape(nz, nx), cmap=cm.jet)
plt.title('Temperature, K')
plt.colorbar()

plt.figure(figsize=(15,3))
p_payzone = pressure[m.up_bs : nb-m.bot_bs]
surf = plt.pcolormesh(X, Y, p_payzone.reshape(nz, nx), cmap=cm.jet)
plt.title('Pressure, bar')
plt.colorbar()